# 🎬 B站/小红书视频字幕提取 - Colab版

## 功能
1. **下载视频** - 支持B站、小红书等平台
2. **Whisper 识别** - GPU 加速语音识别
3. **GLM 优化** - 智谱 API 优化字幕
4. **Google Drive 保存** - 结果自动保存到云端

## 1️⃣ 挂载 Google Drive（可选）

In [ ]:
# 挂载 Google Drive 用于保存结果
from google.colab import drive
drive.mount('/content/drive')

# 创建工作目录
!mkdir -p /content/drive/MyDrive/bilisub_output

## 2️⃣ 安装依赖

In [ ]:
# 安装核心依赖
!pip install -q openai-whisper yt-dlp torch tqdm requests

# 检查 GPU
import torch
print(f"\n🔥 GPU 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU 名称: {torch.cuda.get_device_name(0)}")
    print(f"   GPU 显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 3️⃣ 配置 API（可选）

In [ ]:
# 配置智谱 GLM API（用于字幕优化）
# 如果不需要 GLM 优化，可以跳过

ZHIPU_API_KEY = ""  # @param {type:"string"}

import os
if ZHIPU_API_KEY:
    os.environ['ZHIPU_API_KEY'] = ZHIPU_API_KEY
    print("✅ API 密钥已设置")
else:
    print("⚠️ 未设置 API 密钥，将跳过 GLM 优化")

## 4️⃣ 配置参数

In [ ]:
# ============= 配置区域 =============

# 视频链接（支持 B站、小红书等）
VIDEO_URL = ""  # @param {type:"string"}

# Whisper 模型选择
# tiny: 最快 (~1GB), base: 较快 (~1GB), small: 平衡 (~2GB)
# medium: 较准 (~5GB), large: 最准 (~10GB, 需要更大显存)
WHISPER_MODEL = "small"  # @param ["tiny", "base", "small", "medium", "large"]

# 是否启用 GLM 优化
ENABLE_GLM = False  # @param {type:"boolean"}

# 输出目录
OUTPUT_DIR = "/content/bilisub_output"

# =====================================

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"📁 输出目录: {OUTPUT_DIR}")
print(f"🎬 模型: {WHISPER_MODEL}")
print(f"🤖 GLM优化: {'启用' if ENABLE_GLM and ZHIPU_API_KEY else '禁用'}")

## 5️⃣ 运行完整流程

In [ ]:
import os
import sys
import json
import time
import re
import subprocess
from pathlib import Path
from datetime import datetime

import yt_dlp
import whisper
import torch

# ==================== 核心函数 ====================

def sanitize_filename(name):
    """清理文件名"""
    return re.sub(r'[<>:"/\\|?*]', '_', name)[:100]


def download_audio(url, output_dir):
    """下载音频"""
    print("\n" + "="*60)
    print("⬇️ 步骤 1/3: 下载音频")
    print("="*60)
    
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': str(Path(output_dir) / 'audio.%(ext)s'),
        'quiet': True,
        'no_warnings': True,
    }
    
    start = time.time()
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        title = info.get('title', 'unknown')
        duration = info.get('duration', 0)
    
    # 找到下载的音频文件
    audio_files = list(Path(output_dir).glob('audio.*'))
    if not audio_files:
        raise Exception("音频下载失败")
    
    audio_file = audio_files[0]
    print(f"✅ 下载完成: {time.time()-start:.1f}秒")
    print(f"📌 标题: {title}")
    print(f"⏱️ 时长: {duration}秒 ({duration/60:.1f}分钟)")
    
    return audio_file, title, duration


def transcribe_whisper(audio_file, model_size, output_dir):
    """Whisper 语音识别"""
    print("\n" + "="*60)
    print("🎙️ 步骤 2/3: Whisper 语音识别")
    print("="*60)
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"🔧 设备: {device.upper()}")
    print(f"📦 模型: {model_size}")
    print(f"⏳ 加载模型...")
    
    start = time.time()
    
    # 加载模型
    load_start = time.time()
    model = whisper.load_model(model_size, device=device)
    print(f"✅ 模型加载: {time.time()-load_start:.1f}秒")
    
    # 转录
    print("🔊 正在转录...")
    transcribe_start = time.time()
    result = model.transcribe(
        str(audio_file),
        language="zh",
        task="transcribe",
        verbose=False
    )
    transcribe_time = time.time() - transcribe_start
    
    total_time = time.time() - start
    print(f"✅ 转录完成: {transcribe_time:.1f}秒")
    print(f"⏱️ 总耗时: {total_time:.1f}秒")
    
    return result


def save_srt(segments, output_path):
    """保存为 SRT 格式"""
    with open(output_path, 'w', encoding='utf-8') as f:
        for i, seg in enumerate(segments, 1):
            start_time = seg['start']
            end_time = seg['end']
            
            # 转换时间格式
            def format_time(t):
                hours = int(t // 3600)
                minutes = int((t % 3600) // 60)
                seconds = int(t % 60)
                ms = int((t % 1) * 1000)
                return f"{hours:02d}:{minutes:02d}:{seconds:02d},{ms:03d}"
            
            f.write(f"{i}\n")
            f.write(f"{format_time(start_time)} --> {format_time(end_time)}\n")
            f.write(f"{seg['text'].strip()}\n\n")


def optimize_with_glm(segments, api_key, output_dir):
    """使用 GLM 优化字幕"""
    print("\n" + "="*60)
    print("🤖 步骤 3/3: GLM 字幕优化")
    print("="*60)
    
    import requests
    
    BATCH_SIZE = 5
    optimized_segments = []
    
    total_batches = (len(segments) + BATCH_SIZE - 1) // BATCH_SIZE
    
    for i in range(0, len(segments), BATCH_SIZE):
        batch = segments[i:i+BATCH_SIZE]
        batch_num = i // BATCH_SIZE + 1
        
        # 合并文本
        combined_text = '\n'.join([s['text'].strip() for s in batch])
        
        # 构造 prompt
        prompt = f"""请优化以下视频字幕文本：

1. 修正错别字
2. 添加标点符号
3. 改善语句流畅度
4. 保持原意，不添加新内容

原文（{len(batch)}行）：
{combined_text}

优化后（{len(batch)}行，每行一个）："""
        
        try:
            response = requests.post(
                "https://open.bigmodel.cn/api/paas/v4/chat/completions",
                headers={
                    "Authorization": f"Bearer {api_key}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": "glm-4-flash",
                    "messages": [{"role": "user", "content": prompt}],
                    "temperature": 0.3
                },
                timeout=30
            )
            response.raise_for_status()
            result = response.json()
            optimized_text = result['choices'][0]['message']['content'].strip()
            
            # 分割并更新
            opt_lines = optimized_text.split('\n')
            for j, seg in enumerate(batch):
                if j < len(opt_lines):
                    optimized_segments.append({**seg, 'text': opt_lines[j].strip()})
                else:
                    optimized_segments.append(seg)
            
            print(f"   批次 [{batch_num}/{total_batches}] 完成")
            
        except Exception as e:
            print(f"   ⚠️ 批次 [{batch_num}/{total_batches}] 失败: {e}")
            optimized_segments.extend(batch)
    
    return optimized_segments


def process_video(url, model_size, enable_glm, output_dir):
    """完整处理流程"""
    print("\n" + "🔥"*30)
    print("🎬 开始处理视频")
    print("🔥"*30)
    
    total_start = time.time()
    
    # 1. 下载音频
    audio_file, title, duration = download_audio(url, output_dir)
    safe_title = sanitize_filename(title)
    
    # 2. Whisper 转录
    result = transcribe_whisper(audio_file, model_size, output_dir)
    segments = result['segments']
    
    # 保存原始 SRT
    original_srt = Path(output_dir) / f"{safe_title}_original.srt"
    save_srt(segments, original_srt)
    print(f"\n📄 原始字幕: {original_srt.name}")
    
    # 3. GLM 优化（可选）
    if enable_glm and ZHIPU_API_KEY:
        segments = optimize_with_glm(segments, ZHIPU_API_KEY, output_dir)
        
        # 保存优化后的 SRT
        optimized_srt = Path(output_dir) / f"{safe_title}_optimized.srt"
        save_srt(segments, optimized_srt)
        print(f"📄 优化字幕: {optimized_srt.name}")
    
    # 保存纯文本
    txt_file = Path(output_dir) / f"{safe_title}.txt"
    with open(txt_file, 'w', encoding='utf-8') as f:
        f.write(result['text'])
    print(f"📄 纯文本: {txt_file.name}")
    
    total_time = time.time() - total_start
    
    print("\n" + "="*60)
    print("✅ 处理完成！")
    print("="*60)
    print(f"⏱️ 总耗时: {total_time:.1f}秒 ({total_time/60:.1f}分钟)")
    print(f"📊 识别段落数: {len(segments)}")
    print(f"📁 输出目录: {output_dir}")
    
    return {
        'title': title,
        'duration': duration,
        'segments_count': len(segments),
        'total_time': total_time,
        'files': {
            'original_srt': str(original_srt),
            'optimized_srt': str(Path(output_dir) / f"{safe_title}_optimized.srt") if enable_glm else None,
            'txt': str(txt_file)
        }
    }


# ==================== 执行 ====================

if not VIDEO_URL:
    print("❌ 请先设置 VIDEO_URL")
else:
    result = process_video(
        url=VIDEO_URL,
        model_size=WHISPER_MODEL,
        enable_glm=ENABLE_GLM,
        output_dir=OUTPUT_DIR
    )

## 6️⃣ 保存到 Google Drive

In [ ]:
# 复制结果到 Google Drive
import shutil

drive_output = "/content/drive/MyDrive/bilisub_output"

# 复制所有文件
for file in Path(OUTPUT_DIR).glob('*'):
    if file.is_file():
        dest = Path(drive_output) / file.name
        shutil.copy2(file, dest)
        print(f"📁 已复制: {file.name}")

print(f"\n✅ 所有文件已保存到 Google Drive:")
print(f"   {drive_output}")

## 7️⃣ 下载文件到本地

In [ ]:
# 列出所有输出文件
print("📁 生成的文件:\n")
for file in Path(OUTPUT_DIR).glob('*'):
    if file.is_file():
        size = file.stat().st_size / 1024
        print(f"  • {file.name} ({size:.1f} KB)")

# 下载文件
from google.colab import files

# 下载所有 SRT 和 TXT 文件
for file in Path(OUTPUT_DIR).glob('*.srt'):
    files.download(str(file))
for file in Path(OUTPUT_DIR).glob('*.txt'):
    files.download(str(file))

## 📝 使用说明

### 1. 选择运行时
- 顶部菜单: 运行时 → 更改运行时类型
- 硬件加速器: **GPU** (推荐 T4 或 A100)

### 2. 设置参数
- 第 4 个单元格中设置 `VIDEO_URL` (B站/小红书链接)
- 选择 `WHISPER_MODEL` (推荐 `small` 或 `medium`)
- 如需 GLM 优化，设置 `ZHIPU_API_KEY` 和 `ENABLE_GLM`

### 3. 运行
- 按顺序运行所有单元格
- 或点击: 运行时 → 全部运行

### 4. 获取结果
- 自动保存到 Google Drive
- 或手动下载到本地

### ⚠️ 注意事项
- Colab 免费版有会话时间限制（~90分钟）
- 长视频建议用 `medium` 或 `small` 模型
- GPU 显存不足时，使用更小的模型